In [3]:
import time
import requests
from contextlib import contextmanager

# Context manager

- with가 context manager를 진행한다.

In [4]:
@contextmanager
def timeit():
    start_time = int(round(time.time() * 1000))
    yield
    end_time = int(round(time.time() * 1000))
    elapsed = end_time - start_time
    print("Cook took %d ms to run." % elapsed)

In [5]:
def write_file(filename, content):
    print("Starting writing: %s")
    print("File name is %s" % filename)
    time.sleep(3)

    with open(filename, 'w') as f:
        f.write(content)

In [6]:
files = ['hello.txt', 'world.txt']
content = "Hello world!"

In [7]:
with timeit():
    for filename in files:
        write_file(filename, content)

Starting writing: %s
File name is hello.txt
Starting writing: %s
File name is world.txt
Cook took 6003 ms to run.


# 1. Multi-threading

In [8]:
import threading

In [9]:
with timeit():
    threads = []
    
    for filename in files:
 
        # Create workers
        t = threading.Thread(target=write_file, args=(filename, content))
        
        # Go
        t.start()
        
        # list of threads - sometimes need communication between workers.
        threads.append(t)
        
    for t in threads:
        # Wait till it finishes
        t.join()
    print("Done")

Starting writing: %sStarting writing: %s
File name is world.txt

File name is hello.txt
Done
Cook took 3004 ms to run.


# 2. Synchronization Issues and Locks

In [9]:
import time
import threading

In [10]:
num_threads = 5
iterations_in_one_thread = 100
counter = 0

In [11]:
def f():
    global counter
    
    for i in range(iterations_in_one_thread):
        v = counter
        time.sleep(0.000000001)
        v += 1
        counter = v

In [12]:
def run_experiment():
    global counter 
    
    counter = 0
    threads = []
    
    for i in range(num_threads):
        t = threading.Thread(target=f)
        threads.append(t)
        t.start()
    
    for i in threads:
        i.join()
    
    print("Caculated value: %d" % counter)
    print("Expected value: %d" % (num_threads * iterations_in_one_thread))

In [13]:
run_experiment()

Caculated value: 102
Expected value: 500


### 여기에서 에러가 나타난다.

- 각 thread가 자원을 공유하고자 한다.

In [14]:
# Expected
counter = 0

v1 = counter
v1 += 1
counter = v1

v2 = counter
v2 += 1
counter = v2

print(counter)

2


In [15]:
# Really happens
counter = 0

# thread 1 start
v1 = counter
v1 += 1

# thread 2 start
v2 = counter
v2 += 1
counter = v2

# back to thread 1
counter = v1
print(counter)

1


### Syncronization

In [16]:
num_threads = 5
iterations_in_one_thread = 100
counter = 0

In [17]:
lock = threading.Lock()

In [18]:
# lock을 해제 하는 것을 잊지 말자.
def f():
    global counter
    
    for i in range(iterations_in_one_thread):
        lock.acquire() # begin critical part
        v = counter
        time.sleep(0.000000001)
        v += 1
        counter = v
        lock.release() # end critical part

해제를 하지 않으면 다른 thread가 작업을 할 수 없다!

In [19]:
run_experiment()

Caculated value: 500
Expected value: 500


### Context manager로 해결할 수 있다.

In [21]:
def f():
    global counter, lock
    
    for i in range(iterations_in_one_thread):
        with lock:
            # cirital section
            v = counter
            time.sleep(0.000000001)
            v += 1
            counter = v

In [22]:
run_experiment()

Caculated value: 500
Expected value: 500


- You have to figure out which part you don't want breaks in!
